<a href="https://colab.research.google.com/github/sammigachuhi/ml_projects/blob/main/store_sales_ml_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check if using GPU
import tensorflow as tf

if tf.test.gpu_device_name() == "/device:GPU:0":
  print("GPU Active.")
else:
  print("GPU is not active")

GPU Active.


## Dataset Description
In this competition, you will predict sales for the thousands of product families sold at Favorita stores located in Ecuador. The training data includes dates, store and product information, whether that item was being promoted, as well as the sales numbers. Additional files include supplementary information that may be useful in building your models.

## File Descriptions and Data Field Information
### train.csv

The training data, comprising time series of features store_nbr, family, and onpromotion as well as the target sales.
store_nbr identifies the store at which the products are sold.
family identifies the type of product sold.
sales gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
onpromotion gives the total number of items in a product family that were being promoted at a store at a given date.
test.csv

In [2]:
# Import the requisite libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, GridSearchCV
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, VotingRegressor
from joblib import dump, load


In [3]:
# Load the training data
train = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/kaggle_store_sales/train2.csv")
train.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


## Exploratory Data Analysis

In [4]:
# Check if the training data has any missing values
train.isna().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [5]:
# Investigate the rows that have missing values within the `store_nbr` column
for i in range(len(train)):
  if pd.isna(train["store_nbr"].iloc[i]) == True:
    print(train["id"].iloc[i], train["date"].iloc[i], train["store_nbr"].iloc[i])

In [6]:
# Investigate the rows that have missing values within the `family` column
# To adhere to the "Don't Repeat Yourself" principle, we shall create a function that identifies missing values
# in any column

def find_column_missing_values(dataframe_name, column_name):
  """
  This function takes in the dataframe object and the column within that dataframe
  of which you want to find the id and date values for the row with NaN values of the
  specified column.
  dataframe_name: refers to the dataframe you want to investigate
  column_name: refers to the column with missing values. The name of the column should
  be in string format

  Returns the id, date and value of the row with NaN values of the selected column.
  """
  for i in range(len(dataframe_name)):
    if pd.isna(dataframe_name[column_name].iloc[i]) == True:
      print(dataframe_name.id.iloc[i], dataframe_name.date.iloc[i], dataframe_name[column_name].iloc[i])

In [7]:
# Let's test the above function in printing out the missing value in the `family` column
find_column_missing_values(train, "family")

In [8]:
# Let's test the above function in printing out the missing value in the `sales` column
find_column_missing_values(train, "sales")

In [9]:
# Let's test the above function in printing out the missing value in the `onpromotion` column
find_column_missing_values(train, "onpromotion")

In [10]:
# Investigate rows 188340 - 188355
train.iloc[188340:188355, :]

,id,date,store_nbr,family,sales,onpromotion
188340,188340,2013-04-16,43,DELI,116.0,0
188341,188341,2013-04-16,43,EGGS,29.0,0
188342,188342,2013-04-16,43,FROZEN FOODS,16.0,0
188343,188343,2013-04-16,43,GROCERY I,1656.0,0
188344,188344,2013-04-16,43,GROCERY II,3.0,0
188345,188345,2013-04-16,43,HARDWARE,0.0,0
188346,188346,2013-04-16,43,HOME AND KITCHEN I,0.0,0
188347,188347,2013-04-16,43,HOME AND KITCHEN II,0.0,0
188348,188348,2013-04-16,43,HOME APPLIANCES,1.0,0
188349,188349,2013-04-16,43,HOME CARE,0.0,0


In [11]:
len(train)

3000888

It actually loos like the last row is the one with missing values. Let's investigate other characteristics of the `train` dataframe.


In [12]:
# Describe the `train` dataframe
train.describe()

,id,store_nbr,sales,onpromotion
count,3.000888e+06,3.000888e+06,3.000888e+06,3.000888e+06
mean,1.500444e+06,2.750000e+01,3.577757e+02,2.602770e+00
std,8.662819e+05,1.558579e+01,1.101998e+03,1.221888e+01
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
25%,7.502218e+05,1.400000e+01,0.000000e+00,0.000000e+00
50%,1.500444e+06,2.750000e+01,1.100000e+01,0.000000e+00
75%,2.250665e+06,4.100000e+01,1.958473e+02,0.000000e+00
max,3.000887e+06,5.400000e+01,1.247170e+05,7.410000e+02


In [13]:
# Get info of all the `train` columns
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       object 
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 137.4+ MB


NaN values cause problems during data analysis. It is thus suggested that they are filled in with median (the preferred way). However, one of our coumns, the `family` column is composed of string objects. One Hot encoding is one way of categorizing string values with numerical codes. Then, we can be able to fill in the missing value using the median.


## One Hot Encoding


In [14]:
# Get columns of dtype "object"
columns = []
for col in train.columns:
  if train[col].dtype == "object":
    columns.append(col)

print(columns)

['date', 'family']


In [15]:
# Peform One Hot Encoding on `family` column
train_enc = train.copy() # First copy the original dataframe prior to one hot encoding

train_enc["family"] = train["family"].astype("category")

# Check if the `family` column has been converted to category
train_enc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype   
---  ------       -----   
 0   id           int64   
 1   date         object  
 2   store_nbr    int64   
 3   family       category
 4   sales        float64 
 5   onpromotion  int64   
dtypes: category(1), float64(1), int64(3), object(1)
memory usage: 117.3+ MB


In [16]:
# Assign numerical values to the various category in the `family` column
train_enc["family"] = train_enc["family"].cat.codes

# Check if the `family` column has been assigned categories
train_enc["family"].info()

<class 'pandas.core.series.Series'>
RangeIndex: 3000888 entries, 0 to 3000887
Series name: family
Non-Null Count    Dtype
--------------    -----
3000888 non-null  int8 
dtypes: int8(1)
memory usage: 2.9 MB


In [17]:
# View info on the `train_enc` dataframe
train_enc.head()

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,0,0.0,0
1,1,2013-01-01,1,1,0.0,0
2,2,2013-01-01,1,2,0.0,0
3,3,2013-01-01,1,3,0.0,0
4,4,2013-01-01,1,4,0.0,0


In [18]:
train["family"].unique(), train_enc["family"].unique()

(array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
        'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
        'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
        'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
        'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
        'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
        'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
        'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
        'SEAFOOD'], dtype=object),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
       dtype=int8))

In [19]:
# Check if the length of the original `train` dataframe and `train_enc` match
len(train["family"].unique()), len(train_enc["family"].unique())

(33, 33)

In [20]:
# Fill the NaN values in the `store_nbr`, `family`, `sales` and `promotion` columns with median
train_enc.fillna({
    "store_nbr": train_enc["store_nbr"].median(),
    "family": train_enc["family"].median(),
    "sales": train_enc["sales"].median(),
    "onpromotion": train_enc["onpromotion"].median()
}, inplace=True)

# Check if there are now any missing values in `train_enc` dataframe
train_enc.isna().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

In [21]:
# Now investigate if the last row has any missing values
train_enc.iloc[188340:188355, :]

,id,date,store_nbr,family,sales,onpromotion
188340,188340,2013-04-16,43,9,116.0,0
188341,188341,2013-04-16,43,10,29.0,0
188342,188342,2013-04-16,43,11,16.0,0
188343,188343,2013-04-16,43,12,1656.0,0
188344,188344,2013-04-16,43,13,3.0,0
188345,188345,2013-04-16,43,14,0.0,0
188346,188346,2013-04-16,43,15,0.0,0
188347,188347,2013-04-16,43,16,0.0,0
188348,188348,2013-04-16,43,17,1.0,0
188349,188349,2013-04-16,43,18,0.0,0


In [22]:
# Check if `train_enc` has missing values
train_enc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       int8   
 4   sales        float64
 5   onpromotion  int64  
dtypes: float64(1), int64(3), int8(1), object(1)
memory usage: 117.3+ MB


In [23]:
# Check unique values in the `family` column
train_enc["family"].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32],
      dtype=int8)

There are also other `.csv` files accompanying this project. They are `test.csv`, `stores.csv`, `oil.csv`, and `holiday_events.csv`. Let check them and ensure they are free of any missing values.

In [24]:
# Check `test.csv` dataframe
test = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/kaggle_store_sales/test.csv")
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [25]:
# Check if `test` dataframe has missing values
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.1+ MB


In [26]:
# Check `oil` dataframe
oil = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/kaggle_store_sales/oil.csv")
oil.head()

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


In [27]:
# Check if `oil` dataframe has missing values
oil.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   date        1218 non-null   object 
 1   dcoilwtico  1175 non-null   float64
dtypes: float64(1), object(1)
memory usage: 19.2+ KB


In [28]:
# Check `stores` dataframe
stores = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/kaggle_store_sales/stores.csv")
stores.head()

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


In [29]:
# Check if `stores` dataframe has missing values
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


In [30]:
# Check the `holidays_events`
holidays_events = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/data/kaggle_store_sales/holidays_events.csv")
holidays_events.head()

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


In [31]:
# Check if the `holidays_events` dataframe has missing values
holidays_events.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   date         350 non-null    object
 1   type         350 non-null    object
 2   locale       350 non-null    object
 3   locale_name  350 non-null    object
 4   description  350 non-null    object
 5   transferred  350 non-null    bool  
dtypes: bool(1), object(5)
memory usage: 14.1+ KB


In [32]:
# Check length of all dataframes
len(train_enc), len(test), len(oil), len(stores), len(holidays_events)

(3000888, 28512, 1218, 54, 350)

All the dataframes to be used have different lengths and shapes. Some also have missing data, such as the `oil` dataframe.

## Correlation

In [33]:
# Check how the `train_enc` dataframe columns correlate with each other
train_enc.corr(method="pearson").round(2)

<ipython-input-33-175526c3402f>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  train_enc.corr(method="pearson").round(2)


,id,store_nbr,family,sales,onpromotion
id,1.00,0.00,0.00,0.09,0.21
store_nbr,0.00,1.00,0.00,0.04,0.01
family,0.00,0.00,1.00,-0.11,-0.05
sales,0.09,0.04,-0.11,1.00,0.43
onpromotion,0.21,0.01,-0.05,0.43,1.00


In [34]:
# Let's visualize the correlation between several columns
px.imshow(train_enc.corr(method="pearson").round(2),
          text_auto=True,
          title="Correlation Matrix between columns of training store sales data")

<ipython-input-34-97aed3a054c5>:2: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  px.imshow(train_enc.corr(method="pearson").round(2),


From the above correlation figure, the strongest relationships are between the `onpromotion-sales` columns. Without going into deep mathematical exercises, it seems that promotional products impact sales in some way, only that we don't know if positive or negative. Most other column relationships such as `onpromotion-id` have weak correlation.

In [35]:
# We need to add more data to our train_enc such as some columns from the `stores` dataframe
train_enc_stores = pd.merge(left=train_enc, right=stores,
                            how="left",
                            on="store_nbr")

train_enc_stores.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,0,0.0,0,Quito,Pichincha,D,13
1,1,2013-01-01,1,1,0.0,0,Quito,Pichincha,D,13
2,2,2013-01-01,1,2,0.0,0,Quito,Pichincha,D,13
3,3,2013-01-01,1,3,0.0,0,Quito,Pichincha,D,13
4,4,2013-01-01,1,4,0.0,0,Quito,Pichincha,D,13


In [36]:
# Can date affect sales? Maybe. It seems large shopping occurs on holidays, maybe not.
# Let's join to the `train_enc_stores` dataframe based on date.
train_enc_stores_dates = pd.merge(left=train_enc_stores,
                                  right=holidays_events,
                                  how="left",
                                  on="date")

train_enc_stores_dates.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred
0,0,2013-01-01,1,0,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
1,1,2013-01-01,1,1,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
2,2,2013-01-01,1,2,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
3,3,2013-01-01,1,3,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False
4,4,2013-01-01,1,4,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False


In [37]:
# We are told that Ecuador is prone to oil shocks. Oil shocks have an almost direct effect on inflation, and thus purchasing power
train_enc_stores_dates_oil = pd.merge(left=train_enc_stores_dates,
                                      right=oil,
                                      how="left",
                                      on="date")

train_enc_stores_dates_oil.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred,dcoilwtico
0,0,2013-01-01,1,0,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN
1,1,2013-01-01,1,1,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN
2,2,2013-01-01,1,2,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN
3,3,2013-01-01,1,3,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN
4,4,2013-01-01,1,4,0.0,0,Quito,Pichincha,D,13,Holiday,National,Ecuador,Primer dia del ano,False,NaN


In [38]:
# Now let's check if we have missing data values in our dataframe `train_enc_stores_dates_oil` that combines
# the columns from `stores`, `holidays_events` and `oil` dataframes
train_enc_stores_dates_oil.isna().sum()

id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
city                 0
state                0
type_x               0
cluster              0
type_y         2551824
locale         2551824
locale_name    2551824
description    2551824
transferred    2551824
dcoilwtico      955152
dtype: int64

In [39]:
len(train_enc_stores_dates_oil), (len(train_enc_stores_dates_oil) - len(train_enc)) # find length of the combined dataframe and difference of its length vs original

(3054348, 53460)

A difference of 53460 rows. Those are too many to discard, we result to using the median value to fill in the missing values.

In [40]:
# Check the dtype of columns in `train_enc_stores_dates` dataframe
train_enc_stores_dates_oil.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3054348 entries, 0 to 3054347
Data columns (total 16 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       int8   
 4   sales        float64
 5   onpromotion  int64  
 6   city         object 
 7   state        object 
 8   type_x       object 
 9   cluster      int64  
 10  type_y       object 
 11  locale       object 
 12  locale_name  object 
 13  description  object 
 14  transferred  object 
 15  dcoilwtico   float64
dtypes: float64(2), int64(4), int8(1), object(9)
memory usage: 375.8+ MB


In [41]:
# Create a copy of `train_enc_stores_dates_oil` dataframe whose columns will be one hot encoded
train_enc_stores_dates_oil_cat = train_enc_stores_dates_oil.copy()

In [42]:
# Create a tool that extracts columns of `object` type
columns = []

for col in train_enc_stores_dates_oil_cat.columns:
    if train_enc_stores_dates_oil_cat[col].dtype == "object":
      columns.append(col)

print(columns)

['date', 'city', 'state', 'type_x', 'type_y', 'locale', 'locale_name', 'description', 'transferred']


In [43]:
print(columns)

['date', 'city', 'state', 'type_x', 'type_y', 'locale', 'locale_name', 'description', 'transferred']


In [44]:
# Remove the `date` column
columns.remove("date")
print(columns)

['city', 'state', 'type_x', 'type_y', 'locale', 'locale_name', 'description', 'transferred']


In [45]:
# Create a tool that categorizes the values of the object type columns
def convert_to_categories(dataframe):
  for col in columns:
    # Convert the column values to be categories
    dataframe[col] = dataframe[col].astype("category")
    # Assign numerical values to the various categorical codes in each column
    dataframe[col] = dataframe[col].cat.codes

  return dataframe.info() # Return the info on the various columns of our one hot encoded dataframe

In [46]:
# One Hot Encode all the object type columns in `train_enc_stores_dates_oil` dataframe
convert_to_categories(train_enc_stores_dates_oil_cat)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3054348 entries, 0 to 3054347
Data columns (total 16 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       int8   
 4   sales        float64
 5   onpromotion  int64  
 6   city         int8   
 7   state        int8   
 8   type_x       int8   
 9   cluster      int64  
 10  type_y       int8   
 11  locale       int8   
 12  locale_name  int8   
 13  description  int8   
 14  transferred  int8   
 15  dcoilwtico   float64
dtypes: float64(2), int64(4), int8(9), object(1)
memory usage: 212.6+ MB


In [47]:
train_enc_stores_dates_oil_cat.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred,dcoilwtico
0,0,2013-01-01,1,0,0.0,0,18,12,3,13,3,1,4,50,0,NaN
1,1,2013-01-01,1,1,0.0,0,18,12,3,13,3,1,4,50,0,NaN
2,2,2013-01-01,1,2,0.0,0,18,12,3,13,3,1,4,50,0,NaN
3,3,2013-01-01,1,3,0.0,0,18,12,3,13,3,1,4,50,0,NaN
4,4,2013-01-01,1,4,0.0,0,18,12,3,13,3,1,4,50,0,NaN


In [48]:
# Create a tool that compares the values in the original `train_enc_stores_dates_oil` dataframe vs
# values in the `train_enc_stores_dates_oil_cat` dataframe

def unique_column_values(dataframe1, dataframe2):
  for col in columns:
    for column_name in dataframe1.columns:
      if column_name == col:
        print("\nOriginal Dataframe")
        print(f"Column: {column_name}\n Unique Values: {dataframe1[column_name].unique()}")
    for column_name in dataframe2.columns:
      if column_name == col:
        print("\nOne Hot Encoded Dataframe")
        print(f"Column: {column_name}\n Unique Values: {dataframe2[column_name].unique()}")

In [49]:
# Compare unique values
unique_column_values(train_enc_stores_dates_oil, train_enc_stores_dates_oil_cat)


Original Dataframe
Column: city
 Unique Values: ['Quito' 'Cayambe' 'Latacunga' 'Riobamba' 'Ibarra' 'Santo Domingo'
 'Guaranda' 'Puyo' 'Ambato' 'Guayaquil' 'Salinas' 'Daule' 'Babahoyo'
 'Quevedo' 'Playas' 'Libertad' 'Cuenca' 'Loja' 'Machala' 'Esmeraldas'
 'Manta' 'El Carmen']

One Hot Encoded Dataframe
Column: city
 Unique Values: [18  2 10 19  9 21  7 16  0  8 20  4  1 17 15 11  3 12 13  6 14  5]

Original Dataframe
Column: state
 Unique Values: ['Pichincha' 'Cotopaxi' 'Chimborazo' 'Imbabura'
 'Santo Domingo de los Tsachilas' 'Bolivar' 'Pastaza' 'Tungurahua'
 'Guayas' 'Santa Elena' 'Los Rios' 'Azuay' 'Loja' 'El Oro' 'Esmeraldas'
 'Manabi']

One Hot Encoded Dataframe
Column: state
 Unique Values: [12  3  2  7 14  1 11 15  6 13  9  0  8  4  5 10]

Original Dataframe
Column: type_x
 Unique Values: ['D' 'C' 'B' 'E' 'A']

One Hot Encoded Dataframe
Column: type_x
 Unique Values: [3 2 1 4 0]

Original Dataframe
Column: type_y
 Unique Values: ['Holiday' nan 'Work Day' 'Additional' 'Event' 'Tr

In [50]:
# Find missing values in `train_enc_stores_dates_oil_cat` dataframe
train_enc_stores_dates_oil_cat.isna().sum()

id                  0
date                0
store_nbr           0
family              0
sales               0
onpromotion         0
city                0
state               0
type_x              0
cluster             0
type_y              0
locale              0
locale_name         0
description         0
transferred         0
dcoilwtico     955152
dtype: int64

Even though we can see all columns except for `dcoilwtico` in `train_enc_stores_dates_oil_cat` dataframe are free of missing values, from our `unique_column_values` function above we can see that `nan` values have been replaced with `-1` in the `train_enc_stores_dates_oil_cat` dataframe.

Let replace the `-1` values with the median values.



In [51]:
for col in columns:
  for column_name in train_enc_stores_dates_oil_cat.columns:
    if column_name == col:
      print(f"Median of {column_name}")
      print(f"{train_enc_stores_dates_oil_cat[column_name].median()}")
      print(f"Mean of {column_name}")
      print(f"{train_enc_stores_dates_oil_cat[column_name].mean()}")

Median of city
14.0
Mean of city
12.555555555555555
Median of state
10.0
Mean of state
8.796296296296296
Median of type_x
2.0
Mean of type_x
2.0
Median of type_y
-1.0
Mean of type_y
-0.43348891481913654
Median of locale
-1.0
Mean of locale
-0.7287047841306884
Median of locale_name
-1.0
Mean of locale_name
0.39964994165694284
Median of description
-1.0
Mean of description
5.037922987164527
Median of transferred
-1.0
Mean of transferred
-0.8302217036172695


In [52]:
# Replace the -1 values with median

for col in columns:
  for column_name in train_enc_stores_dates_oil_cat.columns:
    if column_name == col:
      train_enc_stores_dates_oil_cat[column_name] = train_enc_stores_dates_oil_cat[column_name].replace(-1, train_enc_stores_dates_oil_cat[column_name].median())


In [53]:
# Find info on `train_enc_stores_dates_oil_cat` dataframe
train_enc_stores_dates_oil_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3054348 entries, 0 to 3054347
Data columns (total 16 columns):
 #   Column       Dtype  
---  ------       -----  
 0   id           int64  
 1   date         object 
 2   store_nbr    int64  
 3   family       int8   
 4   sales        float64
 5   onpromotion  int64  
 6   city         int8   
 7   state        int8   
 8   type_x       int8   
 9   cluster      int64  
 10  type_y       int8   
 11  locale       int8   
 12  locale_name  int8   
 13  description  int8   
 14  transferred  int8   
 15  dcoilwtico   float64
dtypes: float64(2), int64(4), int8(9), object(1)
memory usage: 212.6+ MB


In [54]:
len(train_enc_stores_dates_oil_cat)

3054348

In [55]:
# Remove the null values from the `train_enc_stores_dates_oil_cat` dataframe
train_enc_stores_dates_oil_cat["dcoilwtico"].fillna(train_enc_stores_dates_oil_cat["dcoilwtico"].median(), inplace=True)

train_enc_stores_dates_oil_cat.isna().sum()

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
city           0
state          0
type_x         0
cluster        0
type_y         0
locale         0
locale_name    0
description    0
transferred    0
dcoilwtico     0
dtype: int64

In [56]:
# Change the `type_x` and `type_y` columns to `type_store` and `type_holidays` columns
train_enc_stores_dates_oil_cat.rename(columns={"type_x": "type_stores", "type_y": "type_holidays"}, inplace=True)
train_enc_stores_dates_oil_cat.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_stores,cluster,type_holidays,locale,locale_name,description,transferred,dcoilwtico
0,0,2013-01-01,1,0,0.0,0,18,12,3,13,3,1,4,50,0,53.43
1,1,2013-01-01,1,1,0.0,0,18,12,3,13,3,1,4,50,0,53.43
2,2,2013-01-01,1,2,0.0,0,18,12,3,13,3,1,4,50,0,53.43
3,3,2013-01-01,1,3,0.0,0,18,12,3,13,3,1,4,50,0,53.43
4,4,2013-01-01,1,4,0.0,0,18,12,3,13,3,1,4,50,0,53.43


In [57]:
# # Convert the time object `date` column to time objects
train_enc_stores_dates_oil_cat["date"] = pd.to_datetime(train_enc_stores_dates_oil_cat["date"]).dt.strftime("%Y%m%d")

# # Check if `date` column is now float
train_enc_stores_dates_oil_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3054348 entries, 0 to 3054347
Data columns (total 16 columns):
 #   Column         Dtype  
---  ------         -----  
 0   id             int64  
 1   date           object 
 2   store_nbr      int64  
 3   family         int8   
 4   sales          float64
 5   onpromotion    int64  
 6   city           int8   
 7   state          int8   
 8   type_stores    int8   
 9   cluster        int64  
 10  type_holidays  int8   
 11  locale         int8   
 12  locale_name    int8   
 13  description    int8   
 14  transferred    int8   
 15  dcoilwtico     float64
dtypes: float64(2), int64(4), int8(9), object(1)
memory usage: 212.6+ MB


In [58]:
# Check if the `date` column has been transformed into time object and hyphens removed
train_enc_stores_dates_oil_cat.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_stores,cluster,type_holidays,locale,locale_name,description,transferred,dcoilwtico
0,0,20130101,1,0,0.0,0,18,12,3,13,3,1,4,50,0,53.43
1,1,20130101,1,1,0.0,0,18,12,3,13,3,1,4,50,0,53.43
2,2,20130101,1,2,0.0,0,18,12,3,13,3,1,4,50,0,53.43
3,3,20130101,1,3,0.0,0,18,12,3,13,3,1,4,50,0,53.43
4,4,20130101,1,4,0.0,0,18,12,3,13,3,1,4,50,0,53.43


In [59]:
# Convert the `date` column to integers
train_enc_stores_dates_oil_cat["date"] = train_enc_stores_dates_oil_cat["date"].astype(int)
train_enc_stores_dates_oil_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3054348 entries, 0 to 3054347
Data columns (total 16 columns):
 #   Column         Dtype  
---  ------         -----  
 0   id             int64  
 1   date           int64  
 2   store_nbr      int64  
 3   family         int8   
 4   sales          float64
 5   onpromotion    int64  
 6   city           int8   
 7   state          int8   
 8   type_stores    int8   
 9   cluster        int64  
 10  type_holidays  int8   
 11  locale         int8   
 12  locale_name    int8   
 13  description    int8   
 14  transferred    int8   
 15  dcoilwtico     float64
dtypes: float64(2), int64(5), int8(9)
memory usage: 212.6 MB


## Training the data

In [60]:
# Check the `train_enc_stores_dates_oil_cat` dataset
train_enc_stores_dates_oil_cat.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_stores,cluster,type_holidays,locale,locale_name,description,transferred,dcoilwtico
0,0,20130101,1,0,0.0,0,18,12,3,13,3,1,4,50,0,53.43
1,1,20130101,1,1,0.0,0,18,12,3,13,3,1,4,50,0,53.43
2,2,20130101,1,2,0.0,0,18,12,3,13,3,1,4,50,0,53.43
3,3,20130101,1,3,0.0,0,18,12,3,13,3,1,4,50,0,53.43
4,4,20130101,1,4,0.0,0,18,12,3,13,3,1,4,50,0,53.43


In [61]:
# Let's visualize the correlation between several columns
px.imshow(train_enc_stores_dates_oil_cat.corr(method="pearson").round(2),
          text_auto=True,
          title="Correlation Matrix between original training sales data, and joined columns from stores, oil and holiday dataframes",
          width=1800,
          height=800)

From the correlation results arising from our expanded table, we can see that theire is a strong positive relationship among the columns from the `holidays` dataframe. These are the `type_holidays`, `locale`, `locale_name`, `description`, and `transferred` columns.

Surprisingly, the columns from the `stores` dataframe, that is the `city`, `state`, `type_stores` and `cluster` columns have negative correlation and no correlation at all with other columns from the original `train` dataframe and `oil`dataframe. The same case applies to the `dcoilwtico` column from the original `oil` dataframe. It has weak to zero correlation to all columns in the expanded `train_enc_stores_dates_oil_cat` dataframe.

In [62]:
# Create the data, the features "X" and the target "y"
X = train_enc_stores_dates_oil_cat.drop(["sales"], axis=1)
y = train_enc_stores_dates_oil_cat["sales"]

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=.2,
                                                    random_state=42)

len(X_train), len(X_test), len(y_train), len(y_test)

(2443478, 610870, 2443478, 610870)

In [63]:
X.head() # Check the `X` dataframe

,id,date,store_nbr,family,onpromotion,city,state,type_stores,cluster,type_holidays,locale,locale_name,description,transferred,dcoilwtico
0,0,20130101,1,0,0,18,12,3,13,3,1,4,50,0,53.43
1,1,20130101,1,1,0,18,12,3,13,3,1,4,50,0,53.43
2,2,20130101,1,2,0,18,12,3,13,3,1,4,50,0,53.43
3,3,20130101,1,3,0,18,12,3,13,3,1,4,50,0,53.43
4,4,20130101,1,4,0,18,12,3,13,3,1,4,50,0,53.43


In [64]:
# X["dcoilwtico"][50000] # Play around with index locations to identify where the `dcoilwtico` change from 53.43 to at least a different value

In [65]:
# # Fit various models to the data
models = []
models.append(("LinearRegression", LinearRegression()))
models.append(("Ridge", Ridge()))
models.append(("Lasso", Lasso()))
models.append(("DecisionTree", DecisionTreeRegressor()))
models.append(("KNN", KNeighborsRegressor()))

# Boosting Algorithms
models.append(("RandomForestRegressor", RandomForestRegressor()))
models.append(("GradientBoostingRegressor", GradientBoostingRegressor()))
models.append(("AdaBoostRegressor", AdaBoostRegressor()))
# models.append(("VotingRegressor", VotingRegressor()))

# Evaluate each model in turn
results = []
names = []

for name, model in models:
  # The `cross_val_score` will fit each model in the `models` list to the data
  cv_results = cross_val_score(model, X_train[:50000], y_train[:50000], cv=5, scoring="r2") # used 50,000 rows to save on time
  # Append results
  results.append(cv_results)
  # Append name
  names.append(name)
  # Print the average of each cross validated model
  print(f"Model: {name}, Mean R2 Score: {cv_results.mean()}")

Model: LinearRegression, Mean R2 Score: 0.21796710702663016
Model: Ridge, Mean R2 Score: 0.2179672357662723
Model: Lasso, Mean R2 Score: 0.2179110806419839
Model: DecisionTree, Mean R2 Score: 0.790721149587467
Model: KNN, Mean R2 Score: -0.14117876925668332
Model: RandomForestRegressor, Mean R2 Score: 0.880349150484918
Model: GradientBoostingRegressor, Mean R2 Score: 0.7130212185557065
Model: AdaBoostRegressor, Mean R2 Score: -0.11557486179699991


In [66]:
# Create a dataframe containing the results of the cross validation values for each model
df = pd.DataFrame(results, index=names)
df

,0,1,2,3,4
LinearRegression,0.217421,0.212676,0.226206,0.219842,0.213691
Ridge,0.217421,0.212677,0.226206,0.219841,0.213692
Lasso,0.217278,0.212850,0.225875,0.219670,0.213883
DecisionTree,0.798891,0.760255,0.818183,0.771045,0.805231
KNN,-0.144303,-0.164629,-0.156780,-0.118204,-0.121978
RandomForestRegressor,0.885835,0.851360,0.897865,0.890112,0.876575
GradientBoostingRegressor,0.728579,0.700745,0.706936,0.719137,0.709710
AdaBoostRegressor,-0.437299,-0.327747,0.157480,0.075602,-0.045910


In [67]:
# Copy the index values to a different column
df["Names"] = df.index
df

,0,1,2,3,4,Names
LinearRegression,0.217421,0.212676,0.226206,0.219842,0.213691,LinearRegression
Ridge,0.217421,0.212677,0.226206,0.219841,0.213692,Ridge
Lasso,0.217278,0.212850,0.225875,0.219670,0.213883,Lasso
DecisionTree,0.798891,0.760255,0.818183,0.771045,0.805231,DecisionTree
KNN,-0.144303,-0.164629,-0.156780,-0.118204,-0.121978,KNN
RandomForestRegressor,0.885835,0.851360,0.897865,0.890112,0.876575,RandomForestRegressor
GradientBoostingRegressor,0.728579,0.700745,0.706936,0.719137,0.709710,GradientBoostingRegressor
AdaBoostRegressor,-0.437299,-0.327747,0.157480,0.075602,-0.045910,AdaBoostRegressor


In [68]:
# In order to visualize a boxplot of the cross validation results, the dataframe needs
# to be melted to long format
df_reshaped = pd.melt(df, id_vars=["Names"], value_vars=[0, 1, 2, 3, 4],
                      value_name="Cross Validation Scores")

df_reshaped

,Names,variable,Cross Validation Scores
0,LinearRegression,0,0.217421
1,Ridge,0,0.217421
2,Lasso,0,0.217278
3,DecisionTree,0,0.798891
4,KNN,0,-0.144303
5,RandomForestRegressor,0,0.885835
6,GradientBoostingRegressor,0,0.728579
7,AdaBoostRegressor,0,-0.437299
8,LinearRegression,1,0.212676
9,Ridge,1,0.212677


In [69]:
# Create box plot showing cross validation scores for our models
box = px.box(
    df_reshaped,
    x="Names",
    y="Cross Validation Scores",
    color="Names",
    title="R2 Scores for Regression Algorithms fitted on Store Sales Data"
)

box

As has been the case in various scenarios, RandomForest is a sturdy algorithm for performing both classification and regression calculations. In our case, RandomForesRegressor() has the highest R2 scores of 0.88, followed by DecisionTreeRegressor and GradientBoostingRegressor at 0.79 and 0.71 R2 scores respectively.

It should be noted the R2 scores were performed on data of just 50,000 rows to save on time, compared to the over 3, 000, 000 rows existent on our dataset.

## Parameter Tuning using GridSearchCV

In [70]:
# %%timeit
# # Set up random seed
# np.random.seed = 42

# # Create parameters
# param_grid = {
#     "n_estimators": [100, 300, 400, 500],
#     "max_depth": [3, 6, 9],
# }

# # Set up GridSearchCV
# grid_search_rfreg = GridSearchCV(estimator=RandomForestRegressor(),
#                                  param_grid=param_grid,
#                                  cv=5,
#                                  n_jobs=-1)

# # Fit the the GridSearchCV on a subset of 50, 000 rows to save on time
# grid_search_rfreg.fit(X_train[:50000], y_train[:50000])

In [71]:
# # Identify best parameters and scores
# print("Best hyperparameters for GridSearchCV RandomForestRegressor:\n", grid_search_rfreg.best_params_)

# # Identify best scores
# print("Best scores for GridSearchCV RandomForestRegressor:\n", grid_search_rfreg.best_score_)

If commented out, here were the `best_params` and `best_score`:

```python
Best hyperparameters for GridSearchCV RandomForestRegressor:
 {'max_depth': 9, 'n_estimators': 400}
Best scores for GridSearchCV RandomForestRegressor:
 0.830826063896572
 ```

Based on our GridSearchCV on RandomForesRegressor, our best_params will yield a score of 0.83. We would have wished to include more hyperparameters in the GridSearchCV, but computing resources and time do not allow.

In [72]:
# Create a RandomForestRegressor model that picks the best parameters
best_rfreg = RandomForestRegressor(
    n_estimators=400,
    max_depth=9
)

best_rfreg

RandomForestRegressor(max_depth=9, n_estimators=400)

In [73]:
# Save the GridSearchCV RandomForestRegressor Model
dump(best_rfreg, "/content/drive/MyDrive/Colab Notebooks/data/kaggle_store_sales/best_rfreg.joblib")


['/content/drive/MyDrive/Colab Notebooks/data/kaggle_store_sales/best_rfreg.joblib']

In [74]:
# Load the saved GridSearchCV RandomForestRegressor model
best_rfreg = load("/content/drive/MyDrive/Colab Notebooks/data/kaggle_store_sales/best_rfreg.joblib")

## Performing Prediction using GridSearchCV Tuned Hyperparamters

In [75]:
%%timeit
# Fit to just 50, 000 rows of the training data in order to save computing resources and time
best_rfreg.fit(X_train[:50000], y_train[:50000])


55.1 s ± 575 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [76]:
# Perform predictions on the entire dataset
y_preds = best_rfreg.predict(X_test[:50000])

In [77]:
# Calculate r2_score of the sales predictions vs the existing predictions
r2_score(y_test[:50000], y_preds)

0.8307900592621489

An R2_score of 0.83 is not too bad nor is it too good. Nevertheless, we shall proceed to peforming predictions on the entire `test` dataframe.

## Using the test data


In [78]:
# Copy the test data
test = test.copy()
test.head()

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


In [79]:
# Length of test
len(test), test.shape

(28512, (28512, 5))

In [80]:
# Join the `stores` dataframe to the `test` dataframe
test_stores = pd.merge(
    left=test,
    right=stores,
    how="left",
    on="store_nbr"
)

test_stores.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13


In [81]:
# Find length of the `test_stores` dataframe
len(test_stores)

28512

In [82]:
# Find any missing values
test_stores.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
 5   city         28512 non-null  object
 6   state        28512 non-null  object
 7   type         28512 non-null  object
 8   cluster      28512 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 2.2+ MB


No missing values. Thankfully so and quite impressive.

In [83]:
# Join the holidays_events dataframe to `test_stores` dataframe
test_stores_dates = pd.merge(
    left=test_stores,
    right=holidays_events,
    how="left",
    on="date"
)

test_stores_dates.head()

,id,date,store_nbr,family,onpromotion,city,state,type_x,cluster,type_y,locale,locale_name,description,transferred
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN


In [84]:
# Find out length of the `test_stores_dates` dataframe
len(test_stores_dates)

28512

In [85]:
# Check if there is any missing data (we expect quite a lot)
test_stores_dates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           28512 non-null  int64 
 1   date         28512 non-null  object
 2   store_nbr    28512 non-null  int64 
 3   family       28512 non-null  object
 4   onpromotion  28512 non-null  int64 
 5   city         28512 non-null  object
 6   state        28512 non-null  object
 7   type_x       28512 non-null  object
 8   cluster      28512 non-null  int64 
 9   type_y       1782 non-null   object
 10  locale       1782 non-null   object
 11  locale_name  1782 non-null   object
 12  description  1782 non-null   object
 13  transferred  1782 non-null   object
dtypes: int64(4), object(10)
memory usage: 3.3+ MB


In [86]:
# Change `type_x` column name to `type_stores`
# Change `type_y` column name to `type_holidays`
test_stores_dates.rename(columns={"type_x": "type_stores", "type_y": "type_holidays"},
                         inplace=True)
test_stores_dates.head()

,id,date,store_nbr,family,onpromotion,city,state,type_stores,cluster,type_holidays,locale,locale_name,description,transferred
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN


In [87]:
# Join the `oil` dataframe
test_stores_dates_oil = pd.merge(
    left=test_stores_dates,
    right=oil,
    how="left",
    on="date"
)

test_stores_dates_oil.head()

,id,date,store_nbr,family,onpromotion,city,state,type_stores,cluster,type_holidays,locale,locale_name,description,transferred,dcoilwtico
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8


In [88]:
# Check length of `test_stores_dates_oil` dataframe
len(test_stores_dates_oil), test_stores_dates_oil.shape

(28512, (28512, 15))

In [89]:
# Check missing data (it will be alot)
test_stores_dates_oil.isna().sum()

id                   0
date                 0
store_nbr            0
family               0
onpromotion          0
city                 0
state                0
type_stores          0
cluster              0
type_holidays    26730
locale           26730
locale_name      26730
description      26730
transferred      26730
dcoilwtico        7128
dtype: int64

## One Hot Encoding the test data

In [90]:
# Find columns of dtype object
# Get columns of dtype "object"
columns_test = []
for col in test_stores_dates_oil.columns:
  if test_stores_dates_oil[col].dtype == "object":
    columns_test.append(col)

print(columns_test)

['date', 'family', 'city', 'state', 'type_stores', 'type_holidays', 'locale', 'locale_name', 'description', 'transferred']


In [91]:
# Convert the `date` column to integer
test_stores_dates_oil["date"] = pd.to_datetime(test_stores_dates_oil["date"]).dt.strftime("%Y%m%d")

# # Check if `date` column is now float
test_stores_dates_oil.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             28512 non-null  int64  
 1   date           28512 non-null  object 
 2   store_nbr      28512 non-null  int64  
 3   family         28512 non-null  object 
 4   onpromotion    28512 non-null  int64  
 5   city           28512 non-null  object 
 6   state          28512 non-null  object 
 7   type_stores    28512 non-null  object 
 8   cluster        28512 non-null  int64  
 9   type_holidays  1782 non-null   object 
 10  locale         1782 non-null   object 
 11  locale_name    1782 non-null   object 
 12  description    1782 non-null   object 
 13  transferred    1782 non-null   object 
 14  dcoilwtico     21384 non-null  float64
dtypes: float64(1), int64(4), object(10)
memory usage: 3.5+ MB


In [92]:
# Now convert the `date` column to integer
test_stores_dates_oil["date"] = test_stores_dates_oil["date"].astype(int)
test_stores_dates_oil.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             28512 non-null  int64  
 1   date           28512 non-null  int64  
 2   store_nbr      28512 non-null  int64  
 3   family         28512 non-null  object 
 4   onpromotion    28512 non-null  int64  
 5   city           28512 non-null  object 
 6   state          28512 non-null  object 
 7   type_stores    28512 non-null  object 
 8   cluster        28512 non-null  int64  
 9   type_holidays  1782 non-null   object 
 10  locale         1782 non-null   object 
 11  locale_name    1782 non-null   object 
 12  description    1782 non-null   object 
 13  transferred    1782 non-null   object 
 14  dcoilwtico     21384 non-null  float64
dtypes: float64(1), int64(5), object(9)
memory usage: 3.5+ MB


In [93]:
# View if the `dates` column is integer
test_stores_dates_oil.head()

,id,date,store_nbr,family,onpromotion,city,state,type_stores,cluster,type_holidays,locale,locale_name,description,transferred,dcoilwtico
0,3000888,20170816,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
1,3000889,20170816,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
2,3000890,20170816,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
3,3000891,20170816,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8
4,3000892,20170816,1,BOOKS,0,Quito,Pichincha,D,13,NaN,NaN,NaN,NaN,NaN,46.8


In [94]:
# Find columns of dtype object again
# Get columns of dtype "object"
columns_test = []
for col in test_stores_dates_oil.columns:
  if test_stores_dates_oil[col].dtype == "object":
    columns_test.append(col)

print(columns_test)

['family', 'city', 'state', 'type_stores', 'type_holidays', 'locale', 'locale_name', 'description', 'transferred']


In [95]:
# Copy results of `columns_test` to `columns` which we had used earlier
columns = columns_test.copy()

In [96]:
# Create `test_enc_stores_dates_oil` that has the same data as `test_stores_dates_oil` dataframe
test_enc_stores_dates_oil = test_stores_dates_oil.copy()

In [97]:
# One Hot Encode all the object type columns in `test_enc_stores_dates_oil` dataframe
convert_to_categories(test_enc_stores_dates_oil)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             28512 non-null  int64  
 1   date           28512 non-null  int64  
 2   store_nbr      28512 non-null  int64  
 3   family         28512 non-null  int8   
 4   onpromotion    28512 non-null  int64  
 5   city           28512 non-null  int8   
 6   state          28512 non-null  int8   
 7   type_stores    28512 non-null  int8   
 8   cluster        28512 non-null  int64  
 9   type_holidays  28512 non-null  int8   
 10  locale         28512 non-null  int8   
 11  locale_name    28512 non-null  int8   
 12  description    28512 non-null  int8   
 13  transferred    28512 non-null  int8   
 14  dcoilwtico     21384 non-null  float64
dtypes: float64(1), int64(5), int8(9)
memory usage: 1.8 MB


In [98]:
# Find median and mean of `test_enc_stores_dates_oil` dataframe
for col in columns:
  for column_name in test_enc_stores_dates_oil.columns:
    if column_name == col:
      print(f"Median of {column_name}")
      print(f"{test_enc_stores_dates_oil[column_name].median()}")
      print(f"Mean of {column_name}")
      print(f"{test_enc_stores_dates_oil[column_name].mean()}")

Median of family
16.0
Mean of family
16.0
Median of city
14.0
Mean of city
12.555555555555555
Median of state
10.0
Mean of state
8.796296296296296
Median of type_stores
2.0
Mean of type_stores
2.0
Median of type_holidays
-1.0
Mean of type_holidays
-0.9375
Median of locale
-1.0
Mean of locale
-0.9375
Median of locale_name
-1.0
Mean of locale_name
-0.9375
Median of description
-1.0
Mean of description
-0.9375
Median of transferred
-1.0
Mean of transferred
-0.9375


In [99]:
# Check the one hot encoded dataframe
test_enc_stores_dates_oil.head()

,id,date,store_nbr,family,onpromotion,city,state,type_stores,cluster,type_holidays,locale,locale_name,description,transferred,dcoilwtico
0,3000888,20170816,1,0,0,18,12,3,13,-1,-1,-1,-1,-1,46.8
1,3000889,20170816,1,1,0,18,12,3,13,-1,-1,-1,-1,-1,46.8
2,3000890,20170816,1,2,2,18,12,3,13,-1,-1,-1,-1,-1,46.8
3,3000891,20170816,1,3,20,18,12,3,13,-1,-1,-1,-1,-1,46.8
4,3000892,20170816,1,4,0,18,12,3,13,-1,-1,-1,-1,-1,46.8


In [102]:
# Fill value -1 in every column with the median
for col in test_enc_stores_dates_oil.columns:
  for i in test_enc_stores_dates_oil[col]:
    if i == -1:
      test_enc_stores_dates_oil[col].replace(-1, test_enc_stores_dates_oil[col].median(skipna=True))



In [103]:
# Check data again
test_enc_stores_dates_oil.head()

,id,date,store_nbr,family,onpromotion,city,state,type_stores,cluster,type_holidays,locale,locale_name,description,transferred,dcoilwtico
0,3000888,20170816,1,0,0,18,12,3,13,-1,-1,-1,-1,-1,46.8
1,3000889,20170816,1,1,0,18,12,3,13,-1,-1,-1,-1,-1,46.8
2,3000890,20170816,1,2,2,18,12,3,13,-1,-1,-1,-1,-1,46.8
3,3000891,20170816,1,3,20,18,12,3,13,-1,-1,-1,-1,-1,46.8
4,3000892,20170816,1,4,0,18,12,3,13,-1,-1,-1,-1,-1,46.8


## Perform predictions on test data

In [104]:
# Check missing values
test_enc_stores_dates_oil.isna().sum()

id                  0
date                0
store_nbr           0
family              0
onpromotion         0
city                0
state               0
type_stores         0
cluster             0
type_holidays       0
locale              0
locale_name         0
description         0
transferred         0
dcoilwtico       7128
dtype: int64

In [105]:
# Fill missing values in the `dcoilwtico` column
test_enc_stores_dates_oil.fillna({
    "dcoilwtico": test_enc_stores_dates_oil["dcoilwtico"].median()
}, inplace=True)

test_enc_stores_dates_oil.isna().sum()

id               0
date             0
store_nbr        0
family           0
onpromotion      0
city             0
state            0
type_stores      0
cluster          0
type_holidays    0
locale           0
locale_name      0
description      0
transferred      0
dcoilwtico       0
dtype: int64

In [106]:
# Fit the GridSearchCV RandomForetRegressor to the data
test_enc_stores_dates_oil["sales"] = best_rfreg.predict(test_enc_stores_dates_oil)

In [107]:
#
test_enc_stores_dates_oil.head()

,id,date,store_nbr,family,onpromotion,city,state,type_stores,cluster,type_holidays,locale,locale_name,description,transferred,dcoilwtico,sales
0,3000888,20170816,1,0,0,18,12,3,13,-1,-1,-1,-1,-1,46.8,5.317762
1,3000889,20170816,1,1,0,18,12,3,13,-1,-1,-1,-1,-1,46.8,0.289520
2,3000890,20170816,1,2,2,18,12,3,13,-1,-1,-1,-1,-1,46.8,44.141025
3,3000891,20170816,1,3,20,18,12,3,13,-1,-1,-1,-1,-1,46.8,2852.042101
4,3000892,20170816,1,4,0,18,12,3,13,-1,-1,-1,-1,-1,46.8,0.453627


In [108]:
# Round the values in the sales column to 3 decimal places
test_enc_stores_dates_oil["sales"] = test_enc_stores_dates_oil["sales"].round(3)

test_enc_stores_dates_oil.head()

,id,date,store_nbr,family,onpromotion,city,state,type_stores,cluster,type_holidays,locale,locale_name,description,transferred,dcoilwtico,sales
0,3000888,20170816,1,0,0,18,12,3,13,-1,-1,-1,-1,-1,46.8,5.318
1,3000889,20170816,1,1,0,18,12,3,13,-1,-1,-1,-1,-1,46.8,0.290
2,3000890,20170816,1,2,2,18,12,3,13,-1,-1,-1,-1,-1,46.8,44.141
3,3000891,20170816,1,3,20,18,12,3,13,-1,-1,-1,-1,-1,46.8,2852.042
4,3000892,20170816,1,4,0,18,12,3,13,-1,-1,-1,-1,-1,46.8,0.454


In [109]:
# Save only the `id` and `sales` columns
gachuhi_submission = test_enc_stores_dates_oil[["id", "sales"]]
gachuhi_submission.head()

,id,sales
0,3000888,5.318
1,3000889,0.290
2,3000890,44.141
3,3000891,2852.042
4,3000892,0.454


In [110]:
# Save to google drive
gachuhi_submission.to_csv("/content/drive/MyDrive/Colab Notebooks/data/kaggle_store_sales/gachuhi_submission.csv",
                          index=False)